# Import data& libraries 

In [406]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

In [407]:
path= "/mnt/d/Data/"

X_train = pd.read_csv(os.path.join(path, "train.csv"))
X_train = X_train[X_train.columns[:-1]]
X_train = X_train[X_train.columns[3:]]    #ID, 이미지 경로 제거
y = train_csv["N_category"]
X_test = pd.read_csv(os.path.join(path, "test.csv"))
submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

In [408]:
X_train.head(5)

,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,63,2015-10-23,1,2,1,19.0,2.0,1.0,2.0,2.0,...,1.0,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN
1,51,2015-10-28,1,1,1,22.0,3.0,3.0,3.0,3.0,...,0.0,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN
2,37,2015-10-29,1,2,1,NaN,2.0,NaN,NaN,NaN,...,1.0,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0
3,54,2016-03-08,1,2,1,0.0,3.0,3.0,3.0,3.0,...,0.0,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN
4,57,2015-10-30,1,2,1,8.0,2.0,2.0,3.0,2.0,...,1.0,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN


In [409]:
print(X_train.info())
print(f"\nSize of X_train: {len(X_train)}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   나이                 1000 non-null   int64  
 1   수술연월일              1000 non-null   object 
 2   진단명                1000 non-null   int64  
 3   암의 위치              1000 non-null   int64  
 4   암의 개수              1000 non-null   int64  
 5   암의 장경              931 non-null    float64
 6   NG                 949 non-null    float64
 7   HG                 914 non-null    float64
 8   HG_score_1         908 non-null    float64
 9   HG_score_2         908 non-null    float64
 10  HG_score_3         911 non-null    float64
 11  DCIS_or_LCIS_여부    1000 non-null   int64  
 12  DCIS_or_LCIS_type  126 non-null    float64
 13  T_category         996 non-null    float64
 14  ER                 999 non-null    float64
 15  ER_Allred_score    703 non-null    float64
 16  PR                 999 no

# Fill missing values

In [410]:
X_train = X_train.fillna(X_train.mean())

/tmp/ipykernel_2208/3112579809.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


# Process categorical values

In [411]:
X_train["수술연월일"].head(5)

0    2015-10-23
1    2015-10-28
2    2015-10-29
3    2016-03-08
4    2015-10-30
Name: 수술연월일, dtype: object

In [412]:
X_train["수술연월일"]

0      2015-10-23
1      2015-10-28
2      2015-10-29
3      2016-03-08
4      2015-10-30
          ...    
995    2006-12-22
996    2006-11-17
997    2006-11-10
998    2006-09-22
999    2010-09-01
Name: 수술연월일, Length: 1000, dtype: object

In [413]:
X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])

In [414]:
X_train["year"] = X_train["수술연월일"].dt.year
X_train["month"] = X_train["수술연월일"].dt.month
X_train["day"] = X_train["수술연월일"].dt.day
X_train["day_name"] = X_train["수술연월일"].dt.day_name

In [415]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   나이                 1000 non-null   int64         
 1   수술연월일              1000 non-null   datetime64[ns]
 2   진단명                1000 non-null   int64         
 3   암의 위치              1000 non-null   int64         
 4   암의 개수              1000 non-null   int64         
 5   암의 장경              1000 non-null   float64       
 6   NG                 1000 non-null   float64       
 7   HG                 1000 non-null   float64       
 8   HG_score_1         1000 non-null   float64       
 9   HG_score_2         1000 non-null   float64       
 10  HG_score_3         1000 non-null   float64       
 11  DCIS_or_LCIS_여부    1000 non-null   int64         
 12  DCIS_or_LCIS_type  1000 non-null   float64       
 13  T_category         1000 non-null   float64       
 14  ER       

In [416]:
date_list=[]
for i in range(len(X_train)):
    date_list.append(str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

In [417]:
X_train["수술연월일"] = date_list
X_train = X_train[X_train.columns[:-1]]

추후 위 코드 제거하고 요일 변수 전처리할것

In [418]:
from sklearn.model_selection import train_test_split

In [419]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, stratify=y, test_size=0.2)

# Train vanilla RF 

In [420]:
from sklearn.ensemble import RandomForestClassifier

In [421]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [422]:
model.score(X_valid, y_valid)

0.83

In [423]:
X_test

,ID,img_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,HG,...,ER,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation
0,BC_01_0011,./test_imgs/BC_01_0011.png,55,2015-11-17,2,2,1,23.0,2.0,2.0,...,1.0,8.0,0.0,2.0,5.00,0.0,1.0,NaN,NaN,NaN
1,BC_01_0220,./test_imgs/BC_01_0220.png,43,2020-06-09,4,2,1,13.0,3.0,2.0,...,1.0,4.0,1.0,8.0,8.67,0.0,0.0,NaN,NaN,NaN
2,BC_01_0233,./test_imgs/BC_01_0233.png,76,2020-05-14,1,1,1,NaN,NaN,NaN,...,1.0,6.0,1.0,6.0,NaN,NaN,2.0,NaN,NaN,NaN
3,BC_01_0258,./test_imgs/BC_01_0258.png,58,2020-05-20,1,2,1,1.3,2.0,2.0,...,1.0,7.0,0.0,NaN,21.17,1.0,3.0,NaN,NaN,0.0
4,BC_01_0260,./test_imgs/BC_01_0260.png,56,2020-05-20,1,2,2,15.0,3.0,3.0,...,1.0,8.0,1.0,3.0,20.57,1.0,3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,BC_01_3328,./test_imgs/BC_01_3328.png,61,2009-10-30,1,1,1,18.0,2.0,2.0,...,1.0,5.0,1.0,3.0,NaN,0.0,0.0,NaN,NaN,NaN
246,BC_01_3404,./test_imgs/BC_01_3404.png,42,2009-05-19,1,2,1,20.0,1.0,2.0,...,1.0,5.0,1.0,5.0,NaN,0.0,1.0,NaN,NaN,NaN
247,BC_01_3418,./test_imgs/BC_01_3418.png,37,2009-04-24,1,1,1,17.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN
248,BC_01_3438,./test_imgs/BC_01_3438.png,37,2009-02-06,1,1,1,7.0,1.0,1.0,...,1.0,5.0,1.0,5.0,NaN,0.0,0.0,NaN,NaN,NaN


In [424]:
def preprocess(X_train):
    X_train = X_train[X_train.columns[2:]]
    X_train = X_train.fillna(X_train.mean())    
    X_train["수술연월일"] = pd.to_datetime(X_train["수술연월일"])
    X_train["year"] = X_train["수술연월일"].dt.year
    X_train["month"] = X_train["수술연월일"].dt.month
    X_train["day"] = X_train["수술연월일"].dt.day
    X_train["day_name"] = X_train["수술연월일"].dt.day_name

    date_list=[]
    for i in range(len(X_train)):
        date_list.append(str(X_train["month"][i]).zfill(2) +str(X_train["day"][i]).zfill(2))

    X_train["수술연월일"] = date_list
    X_train = X_train[X_train.columns[:-1]]
    return X_train

In [425]:
X_test = preprocess(X_test)

/tmp/ipykernel_2208/3131094236.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  X_train = X_train.fillna(X_train.mean())


In [430]:
pred = model.predict(X_test)

In [431]:
submission["N_category"] = pred

In [434]:
submission.to_csv("./submission_tabular.csv", index=False)